# Instalação e importação das bibliotecas

In [1]:
!pip install opencv-python-headless numpy tensorflow matplotlib

In [14]:
import os
import cv2
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks

**Atenção:** O aviso acima apenas informa que não foram encontrados drivers cuda e por conta disso será usada a CPU da máquina ao invés da GPU

# Coleta e processamento de dados

In [3]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized

## Balanceamento do DataSet
Para evitar maior peso em um determinado tipo de dado, foi feito um balanceamento do dataset.

Para isso, foi utilizado o método de reamostragem chamado de undersampling, no qual foi selecionado aleatoriamente os dados de uma classe e duplicado até que a quantidade de dados de cada classe se iguale.

In [4]:
dataset_dir = './dataset'
balanced_dataset_dir = './balanced-dataset'

classes = ['healthy', 'kyphosis', 'lordosis', 'kyphosis_and_lordosis']

os.makedirs(balanced_dataset_dir, exist_ok=True)

image_counts = {}
for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    image_counts[cls] = len(os.listdir(cls_path))

max_images = max(image_counts.values())
new_image_counts = {}

for cls in classes:
    os.makedirs(os.path.join(balanced_dataset_dir, cls), exist_ok=True)
    
    images = os.listdir(os.path.join(dataset_dir, cls))
    
    for index, image in enumerate(images):
        new_name = f"{cls}_{index + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image), 
                    os.path.join(balanced_dataset_dir, cls, new_name))
    
    new_image_counts[cls] = len(images)

    current_count = new_image_counts[cls]
    while current_count < max_images:
        image_to_copy = random.choice(images)
        
        new_name = f"{cls}_{current_count + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image_to_copy),
                    os.path.join(balanced_dataset_dir, cls, new_name))
        
        current_count += 1

print("Dataset balanceado criado em:", balanced_dataset_dir)


Dataset balanceado criado em: ./balanced-dataset


### Criando CNN (rede neural convolucional)
Essa função cria um modelo CNN que passa por algumas camadas convolucionais para extrair características das imagens, nas quais foram utilizados as funções:
- ReLU 
	- Função de ativação que permite que a rede neural aprenda padrões complexos
- Pooling
	- Operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes
- Normalização de Batch
	- Normalização de dados de entrada para que a rede neural possa aprender mais rapidamente
- Softmax
	- Função de ativação que converte um vetor de valores reais em uma distribuição de probabilidade

In [5]:
def create_multilabel_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(3, activation='sigmoid')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

def create_2_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

input_shape = (224, 224, 1)

generalist_cnn_model = create_multilabel_cnn(input_shape)
lordosis_cnn_model = create_2_class_cnn(input_shape)
kiphosis_cnn_model = create_2_class_cnn(input_shape)

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)


## Gerador de alterações nas imagens

Para melhorar a amplitude do treinamento, foi utilizado o gerador de alterações nas imagens, que trás alterações em rotação, deslocamento, zoom, brilho e direção.

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    brightness_range = [0.8, 1.2],
    validation_split = 0.3
)

# Classificação multilabel
Usado no modelo generalista para classificar mais de uma classe em um único caso.
Apontando a classe correta para cada caso.
O único caso que difere dos demais modelos é o 'kyphosis_and_lordosis', para isso, foi criado a classificação dessa classe específica para ser tanto o caso de cifose, quanto o caso de lordose.

In [7]:
def get_multilabel_labels(directory):
    labels = []
    for class_name in ['healthy', 'kyphosis', 'lordosis', 'kyphosis_and_lordosis']:
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            for img in os.listdir(class_dir):
                if class_name == 'kyphosis_and_lordosis':
                    labels.append([0, 1, 1])
                elif class_name == 'healthy':
                    labels.append([1, 0, 0])
                elif class_name == 'kyphosis':
                    labels.append([0, 1, 0])
                elif class_name == 'lordosis':
                    labels.append([0, 0, 1])
    return np.array(labels)

# Pré-processamento dos modelos

Especificamente no caso generalista, as classes foram definidas previamente, por isso a função `multilabel_data_generator` foi criada, para fazer corretamente o apontamento da classe 'kyphosis_and_lordosis' para ambas as classes 'kyphosis' e 'lordosis'.

In [8]:
generalist_classes = ['healthy', 'kyphosis', 'lordosis']

def multilabel_data_generator(generator, labels):
    while True:
        data = next(generator)
        batch_size = data.shape[0]

        yield data, labels[:batch_size]
            
initial_generalist_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = None,
    shuffle = True,
    subset = 'training'
)

initial_generalist_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = None,
    shuffle = True,
    subset = 'validation'
)

labels = get_multilabel_labels('./balanced-dataset')

generalist_train_generator = multilabel_data_generator(initial_generalist_train_generator, labels)
generalist_validation_generator = multilabel_data_generator(initial_generalist_validation_generator, labels)

generalist_cnn_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

Found 232 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


Para aprimorar o treinamento do modelo, foi criada as funções `kyphosis_custom_generator` e `lordosis_custom_generator`, que tem como objetivo considerar as imagens da pasta `kyphosis_and_lordosis` para os casos de cifose, no modelo de cifose, e para os casos de lordose, no modelo de lordose.

In [26]:
def kyphosis_custom_generator(data_dir, kiphosis_classes, target_size=(224, 224), batch_size=4, subset='training'):
    # Mapeamento dos diretórios
    healthy_dir = os.path.join(data_dir, 'healthy')
    kyphosis_dir = os.path.join(data_dir, 'kyphosis')
    kyphosis_and_lordosis_dir = os.path.join(data_dir, 'kyphosis_and_lordosis')
    
    # Carregar caminhos de imagens
    healthy_images = [os.path.join(healthy_dir, img) for img in os.listdir(healthy_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_images = [os.path.join(kyphosis_dir, img) for img in os.listdir(kyphosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_and_lordosis_images = [os.path.join(kyphosis_and_lordosis_dir, img) for img in os.listdir(kyphosis_and_lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    
    # Associação das imagens com as classes
    images = healthy_images + kyphosis_images + kyphosis_and_lordosis_images
    labels = [0] * len(healthy_images) + [1] * (len(kyphosis_images) + len(kyphosis_and_lordosis_images))
    labels = to_categorical(labels, num_classes=2)

    # Divisão entre treinamento e validação
    split_index = int(len(images) * 0.8)
    if subset == 'training':
        images, labels = images[:split_index], labels[:split_index]
    else:
        images, labels = images[split_index:], labels[split_index:]

    while True:
        batch_images = []
        batch_labels = []
        
        # Embaralhar as imagens e labels
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        
        for i in indices:
            img = load_img(images[i], target_size=target_size, color_mode='grayscale')  # Alterado para RGB
            img_array = img_to_array(img)
            batch_images.append(img_array)
            batch_labels.append(labels[i])

            if len(batch_images) == batch_size:
                yield np.array(batch_images), np.array(batch_labels)
                batch_images, batch_labels = [], []

        # Gerar batch incompleto se restarem imagens
        if batch_images:
            yield np.array(batch_images), np.array(batch_labels)

def lordosis_custom_generator(data_dir, lordosis_classes, target_size=(224, 224), batch_size=4, subset='training'):
    # Mapeamento dos diretórios
    healthy_dir = os.path.join(data_dir, 'healthy')
    lordosis_dir = os.path.join(data_dir, 'lordosis')
    kyphosis_and_lordosis_dir = os.path.join(data_dir, 'kyphosis_and_lordosis')
    
    # Carregar caminhos de imagens
    healthy_images = [os.path.join(healthy_dir, img) for img in os.listdir(healthy_dir) if img.endswith('.png') or img.endswith('.jpg')]
    lordosis_images = [os.path.join(lordosis_dir, img) for img in os.listdir(lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_and_lordosis_images = [os.path.join(kyphosis_and_lordosis_dir, img) for img in os.listdir(kyphosis_and_lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    
    # Associação das imagens com as classes
    images = healthy_images + lordosis_images + kyphosis_and_lordosis_images
    labels = [0] * len(healthy_images) + [1] * (len(lordosis_images) + len(kyphosis_and_lordosis_images))
    labels = to_categorical(labels, num_classes=2)

    # Divisão entre treinamento e validação
    split_index = int(len(images) * 0.8)
    if subset == 'training':
        images, labels = images[:split_index], labels[:split_index]
    else:
        images, labels = images[split_index:], labels[split_index:]

    while True:
        batch_images = []
        batch_labels = []
        
        # Embaralhar as imagens e labels
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        
        for i in indices:
            img = load_img(images[i], target_size=target_size, color_mode='grayscale')  # Alterado para RGB
            img_array = img_to_array(img)
            batch_images.append(img_array)
            batch_labels.append(labels[i])

            if len(batch_images) == batch_size:
                yield np.array(batch_images), np.array(batch_labels)
                batch_images, batch_labels = [], []

        # Gerar batch incompleto se restarem imagens
        if batch_images:
            yield np.array(batch_images), np.array(batch_labels)


In [32]:
lordosis_classes = ['healthy', 'lordosis']

lordosis_train_generator = lordosis_custom_generator(
    data_dir = './balanced-dataset',
    lordosis_classes = lordosis_classes,
    target_size = (224, 224),
    batch_size = 4,
    subset = 'training'
)

lordosis_validation_generator = lordosis_custom_generator(
    data_dir = './balanced-dataset',
    lordosis_classes = lordosis_classes,
    target_size = (224, 224),
    batch_size = 4,
    subset = 'validation'
)

lordosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [33]:
kiphosis_classes = ['healthy', 'kyphosis']

kiphosis_train_generator = kyphosis_custom_generator(
    data_dir = './balanced-dataset',
    kiphosis_classes = kiphosis_classes,
    target_size = (224, 224),
    batch_size = 4,
    subset = 'training'
)

kiphosis_validation_generator = kyphosis_custom_generator(
    data_dir = './balanced-dataset',
    kiphosis_classes = kiphosis_classes,
    target_size = (224, 224),
    batch_size = 4,
    subset = 'validation'
)

kiphosis_cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinamento do modelo

Como estamos usando uma rede pre-treinada para facilitar o treinamento (a rede já identifica curvas, linhas e padrões mais simples), presisamos limpar a sessão de treinamento antes de treinar novamente.

In [34]:
K.clear_session()

O modelo generalista, por ter uma pre-definição de classes, ele não identifica de forma automática a quantidade de passos por época, para calcular isso, vamos utilizar o método `len()`, que retorna o tamanho de um objeto e dividir pelo tamanho do batch.

Para os demais modelos, a quantidade de passos será basicamente a quantidade de imagens divido pela tamanho do batch.

In [35]:
data_dir = './balanced-dataset'
batch_size = 4 
target_size = (224, 224)

def list_kyphosis_images(data_dir):
    healthy_dir = os.path.join(data_dir, 'healthy')
    kyphosis_dir = os.path.join(data_dir, 'kyphosis')
    kyphosis_and_lordosis_dir = os.path.join(data_dir, 'kyphosis_and_lordosis')

    healthy_images = [os.path.join(healthy_dir, img) for img in os.listdir(healthy_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_images = [os.path.join(kyphosis_dir, img) for img in os.listdir(kyphosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_and_lordosis_images = [os.path.join(kyphosis_and_lordosis_dir, img) for img in os.listdir(kyphosis_and_lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]

    images = healthy_images + kyphosis_images + kyphosis_and_lordosis_images
    labels = [0] * len(healthy_images) + [1] * (len(kyphosis_images) + len(kyphosis_and_lordosis_images))  # 0 para healthy, 1 para kyphosis
    labels = to_categorical(labels, num_classes=2)

    return images, labels

def list_lordosis_images(data_dir):
    healthy_dir = os.path.join(data_dir, 'healthy')
    lordosis_dir = os.path.join(data_dir, 'lordosis')
    kyphosis_and_lordosis_dir = os.path.join(data_dir, 'kyphosis_and_lordosis')

    healthy_images = [os.path.join(healthy_dir, img) for img in os.listdir(healthy_dir) if img.endswith('.png') or img.endswith('.jpg')]
    lordosis_images = [os.path.join(lordosis_dir, img) for img in os.listdir(lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]
    kyphosis_and_lordosis_images = [os.path.join(kyphosis_and_lordosis_dir, img) for img in os.listdir(kyphosis_and_lordosis_dir) if img.endswith('.png') or img.endswith('.jpg')]

    images = healthy_images + lordosis_images + kyphosis_and_lordosis_images
    labels = [0] * len(healthy_images) + [1] * (len(lordosis_images) + len(kyphosis_and_lordosis_images))  # 0 para healthy, 1 para kyphosis
    labels = to_categorical(labels, num_classes=2)

    return images, labels

kyphosis_images, kyphosis_labels = list_kyphosis_images(data_dir)
lordosis_images, lordosis_labels = list_lordosis_images(data_dir)

kyphosis_split_index = int(len(kyphosis_images) * 0.8)
lordosis_split_index = int(len(lordosis_images) * 0.8)

kyphosis_train_images, kyphosis_val_images = kyphosis_images[:kyphosis_split_index], kyphosis_images[kyphosis_split_index:]
lordosis_train_images, lordosis_val_images = lordosis_images[:lordosis_split_index], lordosis_images[lordosis_split_index:]

kyphosis_steps_per_epoch = len(kyphosis_train_images) // batch_size
kyphosis_validation_steps = len(kyphosis_val_images) // batch_size
lordosis_steps_per_epoch = len(lordosis_train_images) // batch_size
lordosis_validation_steps = len(lordosis_val_images) // batch_size

generalist_steps_per_epoch = len(labels) // initial_generalist_train_generator.batch_size
generalist_validation_steps = len(labels) // initial_generalist_validation_generator.batch_size


In [37]:
lordosis_history = lordosis_cnn_model.fit(
    lordosis_train_generator,
    steps_per_epoch = lordosis_steps_per_epoch,
    validation_data = lordosis_validation_generator,
    validation_steps = lordosis_validation_steps,
    epochs = 10,
    callbacks = [lr_scheduler]
)

kiphosis_cnn_model.fit(
    kiphosis_train_generator,
    steps_per_epoch = kyphosis_steps_per_epoch,
    validation_data = kiphosis_validation_generator,
    validation_steps = kyphosis_validation_steps,
    epochs = 10,
    callbacks = [lr_scheduler]
)

generalist_history = generalist_cnn_model.fit(
    generalist_train_generator,
    validation_data = generalist_validation_generator,
    steps_per_epoch = generalist_steps_per_epoch,
    validation_steps = generalist_validation_steps,
    epochs = 10,
    callbacks = [lr_scheduler]
)

Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.9159 - loss: 1.4518 - val_accuracy: 0.6087 - val_loss: 2.5917 - learning_rate: 5.0000e-04
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 46s 946ms/step - accuracy: 0.9065 - loss: 1.4001 - val_accuracy: 0.8913 - val_loss: 1.4694 - learning_rate: 5.0000e-04
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 45s 923ms/step - accuracy: 0.9218 - loss: 1.3334 - val_accuracy: 0.6087 - val_loss: 1.9852 - learning_rate: 5.0000e-04
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 37s 765ms/step - accuracy: 0.9346 - loss: 1.3648 - val_accuracy: 0.9783 - val_loss: 1.3641 - learning_rate: 5.0000e-04
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 38s 781ms/step - accuracy: 0.9684 - loss: 1.2114 - val_accuracy: 0.7826 - val_loss: 1.4922 - learning_rate: 5.0000e-04
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 39s 805ms/step - accuracy: 0.9478 - loss: 1.1516 - val_accuracy: 0.8261 - val_loss: 1.4753 - learning_rate: 5.0000e-04
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 41s 834ms/step - accura

## Porque resultados sempre são diferentes:
- Ao inicializar o modelo os pesos da rede neural são inicializados de forma aleatória, e como o processo de otimização do modelo começa a partir de diferentes pontos cada execução pode levar a resultados diferentes, por isso, cada vez que o código for executado os resultados não serão exatamente os mesmos, mas aproximados.
- Image Augmentation: técnicas de rotação, deslocamento, zoom e etc. aplicadas na imagem para criar novas variações para realizar o treinamento do modelo, sendo também um processo aleatório.
- Shuffling: Durante o treinamento, os dados de treino são embaralhados a cada época. Isso garante que o modelo não aprenda de forma dependente da ordem dos exemplos, mas também pode fazer com que os resultados variem.

# Validação dos modelos

In [40]:
generalist_val_loss, generalist_val_acc = generalist_cnn_model.evaluate(generalist_validation_generator, steps = generalist_validation_steps)
lordosis_val_loss, lordosis_val_acc = lordosis_cnn_model.evaluate(lordosis_validation_generator, steps = lordosis_validation_steps)
kiphosis_val_loss, kiphosis_val_acc = kiphosis_cnn_model.evaluate(kiphosis_validation_generator, steps = kyphosis_validation_steps)

print(f"Validação modelo generalista - Loss: {generalist_val_loss}, Acurácia: {generalist_val_acc}")
print(f"Validação modelo de lordose - Loss: {lordosis_val_loss}, Acurácia: {lordosis_val_acc}")
print(f"Validação modelo de cifose - Loss: {kiphosis_val_loss}, Acurácia: {kiphosis_val_acc}")

82/82 ━━━━━━━━━━━━━━━━━━━━ 51s 629ms/step - accuracy: 1.0000 - loss: 0.0351
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 563ms/step - accuracy: 0.9568 - loss: 0.9531
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 585ms/step - accuracy: 0.6479 - loss: 1.8741
Validação modelo generalista - Loss: 0.0350940078496933, Acurácia: 1.0
Validação modelo de lordose - Loss: 0.948793888092041, Acurácia: 0.97826087474823
Validação modelo de cifose - Loss: 1.7229458093643188, Acurácia: 0.6739130616188049


# Função de predição

In [42]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction
    
healthy_img = './balanced-dataset/healthy/healthy_001.jpg'
lordosis_img = './balanced-dataset/lordosis/lordosis_001.jpg'
kyphosis_img = './balanced-dataset/kyphosis/kyphosis_001.jpg'
kyphosis_and_lordosis_img = './balanced-dataset/kyphosis_and_lordosis/kyphosis_and_lordosis_001.jpg'

generalist_predicted_class, generalist_result = predict_image(healthy_img, generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image(healthy_img, lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image(healthy_img, kiphosis_cnn_model)

print(f'Imagem saudável:')
print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

generalist_predicted_class, generalist_result = predict_image(kyphosis_img, generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image(kyphosis_img, lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image(kyphosis_img, kiphosis_cnn_model)

print(f'\nImagem cifose:')
print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

generalist_predicted_class, generalist_result = predict_image(lordosis_img, generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image(lordosis_img, lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image(lordosis_img, kiphosis_cnn_model)

print(f'\nImagem lordose:')
print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

generalist_predicted_class, generalist_result = predict_image(kyphosis_and_lordosis_img, generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image(kyphosis_and_lordosis_img, lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image(kyphosis_and_lordosis_img, kiphosis_cnn_model)

print(f'\nImagem cifose e lordose:')
print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Imagem saudável:
Predição (modelo generalista): [[9.9966615e-01 2.9946500e-04 3.5025459e-04]]
Classe prevista (modelo generalista): healthy
Predição (modelo de lordose): [[0.18739401 0.812606  ]]
Classe prevista (modelo de lordose): lordosis
Predição (modelo de cifose): [[0.15936992 0.84063005]]
Classe prevista (modelo de cifose): kyphosis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step

Imagem cifose:
Predição (modelo generalista): [[9.9966645e-01 2.9918845e-04 3.4992685e-04]]
Classe prevista (modelo generalista): healthy
Predição (modelo de lordose): [[0.189899 0.810101]]
Classe prevista (modelo de lordose): lordosis
Predição (modelo de cifose): [[0.16025211 0.8397479 ]]
Classe prevista (modelo de cifose): kyphosis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━